# Quantum Basic Arithmetic Operations

In this notebook some circuits for quantum Fourier Transformation (qFT) based operators are experimented with. The idea is to get a feeling how they work.

[1] paper: G. Florio 1 and D. Picca (2014): "Quantum implementation of elementary arithmetic operations"

[2] paper: E. Şahin (2020): "QUANTUM ARITHMETIC OPERATIONS BASED ON QUANTUM FOURIER TRANSFORM ON SIGNED INTEGERS"


## qFT based addition
Based on [2]

1. Adder
2. Subtractor
3. Multiplier


### modular adder
* consider two _signed_ integers a and b stored in two seperate registers of length n and m respectively, with $n >= m$:
  * $|ket a> = |a_1 a_2 ... a_n>$
  * $|ket b> = |b_1 b_2 ... b_m>$
  
* apply qFT on a and its significant information will be in a superposition: $|\phi_1(a)> = \frac{(|0> + e^{2\pi i 0.a_1 a_2 \dots a_n} |1> )}{2^{1/2}}$
  
* Consider a quantum phase gate  $ R_k = \begin{bmatrix} 1 & 0 \\ 0 & e^{2\pi i / 2^k}\end{bmatrix}$

* trace most significant bits -> apply to all qubits

* use inverse qFT to optain most significant bits of $a+b$

In [1]:
import cirq
import numpy as np
import bitstring

Let's defines some helpers.

In [2]:
def simulate(circuit, repetitions=1000):
    result = {}
    simulator = cirq.Simulator()
    results = simulator.simulate(circuit)
    samples = simulator.run(circuit, repetitions=repetitions)
    return samples


class CircuitInitializer(cirq.Circuit):
    
    def __init__(self, number_a, number_b, *contents: 'cirq.OP_TREE', offset=0):
        super().__init__(*contents)
        self.cbits_b = self.get_bit_representation(number_b)
        self.m = len(self.cbits_b)
        self.cbits_a = self.get_bit_representation(number_a, self.m - 1)
        self.n = len(self.cbits_a)
        self.offset = offset
        
        self.register_a = cirq.LineQubit.range(self.offset, self.n + self.offset)
        self.register_b = cirq.LineQubit.range(self.offset + self.n, self.m + self.n + self.offset)
        
        self._init_circuit()

    def _init_circuit(self):
        for i in range(0, self.n):
            if 1 == self.cbits_a[i]:
                self.append(cirq.X(self.register_a[i]))

        for i in range(0, self.m):
            if 1 == self.cbits_b[i]:
                self.append(cirq.X(self.register_b[i]))
    

    def get_bit_representation(self, number, min_length=0):
        bits = bitstring.BitArray(bin=bin(abs(number)))
        while len(bits) < min_length:
            bits = [0] + bits

        if 0 > number:
            bits = ~bits
            bits = bitstring.BitArray(bin=bin(int(bits.bin,2) + int('1',2)))
        else:
            bits = [0] + bits

        return bits


Let's define the modular adder.

In [3]:
class QModularAdder(cirq.Circuit):
    
    def __init__(self, register_a, register_b, *contents: 'cirq.OP_TREE', debug=False):
        if debug:
            print("QModularAdder")
        super().__init__(*contents)
        if debug:
            print("back in QModularAdder")
        self.register_a = register_a
        self.register_b = register_b
        self.n = len(self.register_a)
        self.m = len(self.register_b)
    
    def apply(self):
        self._apply_qft()
        self._apply_modular_adder( self.n, self.m)
        self._apply_inverse_qft()
        self._apply_measurement()
    
    def _apply_qft(self):
        self.append(cirq.qft(*self.register_a, without_reverse=True))
        
    def _apply_modular_adder(self, max_a, max_b):
        for index_a in range(1, max_a+1):
            max_k = min(index_a, max_b)
            self._apply_operator(index_a, max_k)
                
    def _apply_operator(self, index_a, max_k, min_k=1):
        k = min_k
        while k <= max_k:
            self.append(self._rk_gate_operator(k).on(self.register_a[self.n-index_a]).controlled_by(self.register_b[self.m-1-max_k+k]))
            k +=1
    
    def _rk_gate_operator(self, k):
        return cirq.MatrixGate(matrix=np.array([[1, 0], [0, np.exp(2 * np.pi * 1.0j / 2 ** k)]]))

    def _apply_inverse_qft(self):
        self.append(cirq.inverse(cirq.qft(*self.register_a, without_reverse=True)))
    
    def _apply_measurement(self):
        self.append(cirq.measure(*self.register_a, key='result'))

In [4]:
def modular_adding(number_a, number_b, print_circuit=False):
    circuit = CircuitInitializer(number_a, number_b)

    qmadd = QModularAdder(circuit.register_a, circuit.register_b)
    qmadd.apply()
    circuit += qmadd
    
    if print_circuit:
        print(circuit)
    
    samples = simulate(circuit, 1000)
    return samples

Let's put everything together to do some end-to-end testing.
We don't deal with negative integers here.

In [5]:
number_a = 0
number_b = 1
samples = modular_adding(number_a, number_b, True)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' + ',bin(number_b)[2:],'= expected:',bin(number_a + number_b)[2:],'got:',bin(samples.data['result'][0])[2:])


number_a = 2
number_b = 5
samples = modular_adding(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' + ',bin(number_b)[2:],'= expected:',bin(number_a + number_b)[2:],'got:',bin(samples.data['result'][0])[2:])

number_a = 3
number_b = 3
samples = modular_adding(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' + ',bin(number_b)[2:],'= expected:',bin(number_a + number_b)[2:],'got:',bin(samples.data['result'][0])[2:])


                       ┌──────────────────────────────────┐
                                         ┌               ┐    ┌             ┐
0: ───────qft[norev]─────────────────────│ 1.+0.j  0.+0.j│────│1.+0.j 0.+0.j│───qft[norev]^-1───M('result')───
          │                              │ 0.+0.j -1.+0.j│    │0.+0.j 0.+1.j│   │               │
          │                              └               ┘    └             ┘   │               │
          │                              │                    │                 │               │
          │             ┌               ┐│                    │                 │               │
1: ───────#2────────────│ 1.+0.j  0.+0.j│┼────────────────────┼─────────────────#2──────────────M─────────────
                        │ 0.+0.j -1.+0.j││                    │
                        └               ┘│                    │
                        │                │                    │
2: ─────────────────────┼────────────────@──────────────

### non-modular adder 

For the non-modular adder we need an additional ancilla qubit in register a to compensate for the sign of the sum of (a + b).

We can reuse the modular adder by enhancing it to adjust to the additional ancilla qubit.

In [6]:
class QNonModularAdder(QModularAdder):
    def __init__(self, register_a, register_b, *contents: 'cirq.OP_TREE', debug=False):
        if debug:
            print("QNonModularAdder")
        super().__init__(register_a, register_b, *contents)
        if debug:
            print("back in QNonModularAdder")
        self.register_a = [cirq.LineQubit(0)] + self.register_a
        self.n = len(self.register_a)
        
        self.append(cirq.CCNOT(self.register_b[0], self.register_a[1], self.register_a[0]))
        self.append(cirq.CCNOT(self.register_a[1], self.register_b[0], self.register_a[0]))
        
    
    def _apply_modular_adder(self, max_a, max_b):
        super()._apply_modular_adder(max_a-1, max_b)
        index_a = self.n
        max_k = min(index_a, self.m+1)
        min_k = 2
        self._apply_operator(index_a, max_k, min_k)
    

def non_modular_adding(number_a, number_b, print_circuit=False):
    circuit = CircuitInitializer(number_a, number_b, offset=1)

    qnmadd = QNonModularAdder(circuit.register_a, circuit.register_b)
    qnmadd.apply()
    circuit += qnmadd
    
    if print_circuit:
        print(circuit)
    
    samples = simulate(circuit, 1000)
    return samples

In [7]:
number_a = 0
number_b = 1
samples = non_modular_adding(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' + ',bin(number_b)[2:],'= expected:',bin(number_a + number_b)[2:],'got:',bin(samples.data['result'][0])[2:])

number_a = 2
number_b = 5
samples = non_modular_adding(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' + ',bin(number_b)[2:],'= expected:',bin(number_a + number_b)[2:],'got:',bin(samples.data['result'][0])[2:])

number_a = 3
number_b = 3
samples = non_modular_adding(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' + ',bin(number_b)[2:],'= expected:',bin(number_a + number_b)[2:],'got:',bin(samples.data['result'][0])[2:])


# not correct...
number_a = -1
number_b = -2
samples = non_modular_adding(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' + ',bin(number_b)[2:],'= expected:',bin(number_a + number_b)[2:],'got:',bin(samples.data['result'][0])[2:])


Counter({1: 1000})
0  +  1 = expected: 1 got: 1
Counter({7: 1000})
10  +  101 = expected: 111 got: 111
Counter({6: 1000})
11  +  11 = expected: 110 got: 110
Counter({1: 1000})
b1  +  b10 = expected: b11 got: 1


#### Open issues
- how to correctly set up toffoli gate?
- negative integers as input

#### Possible test cases
- Endo-to-end tests: pos + pos integer; neg + pos integer; neg + neg integer; 
- test on circuit structure
- mutation: 
    - insert: additional qubit; additonal rotation
    - remove: qubit; toffoli gates; inverse qFT
    - change: rotation operator
- note: how to test for noise?

## qFT based subtraction
To implement modular subtraction we can enhance the modular adder.

* consider operator $R^{-1} = \begin{bmatrix} 1 & 0 \\ 0 &  e^{-2\pi i / 2^k}\end{bmatrix}$

In [8]:
class QModularSubtractor(QModularAdder):
    def _rk_gate_operator(self, k):
        return cirq.MatrixGate(matrix=np.array([[1, 0], [0, np.exp(-2 * np.pi * 1.0j / 2 ** k)]]))

def modular_subtracting(number_a, number_b, print_circuit=False):
    circuit = CircuitInitializer(number_a, number_b)

    qmsub = QModularSubtractor(circuit.register_a, circuit.register_b)
    qmsub.apply()
    circuit += qmsub
    
    if print_circuit:
        print(circuit)
    
    samples = simulate(circuit, 1000)
    return samples

In [9]:
number_a = 1
number_b = 0
samples = modular_subtracting(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' - ',bin(number_b)[2:],'= expected:',bin(number_a - number_b)[2:],'got:',bin(samples.data['result'][0])[2:])


number_a = 3
number_b = 1
samples = modular_subtracting(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' - ',bin(number_b)[2:],'= expected:',bin(number_a - number_b)[2:],'got:',bin(samples.data['result'][0])[2:])


number_a = 3
number_b = 3
samples = modular_subtracting(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' - ',bin(number_b)[2:],'= expected:',bin(number_a - number_b)[2:],'got:',bin(samples.data['result'][0])[2:])


Counter({1: 1000})
1  -  0 = expected: 1 got: 1
Counter({2: 847, 6: 153})
11  -  1 = expected: 10 got: 10
Counter({0: 1000})
11  -  11 = expected: 0 got: 0


In [10]:
class QNonModularSubtractor(QNonModularAdder):
    print("QNonModularSubtractor")
    def _rk_gate_operator(self, k):
        return cirq.MatrixGate(matrix=np.array([[1, 0], [0, np.exp(-2 * np.pi * 1.0j / 2 ** k)]]))

def non_modular_subtracting(number_a, number_b, print_circuit=False):
    circuit = CircuitInitializer(number_a, number_b, offset=1)

    qnmsub = QNonModularSubtractor(circuit.register_a, circuit.register_b)
    qnmsub.apply()
    circuit += qnmsub
    
    if print_circuit:
        print(circuit)
    
    samples = simulate(circuit, 1000)
    return samples

QNonModularSubtractor


In [11]:
number_a = 1
number_b = 0
samples = non_modular_subtracting(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' - ',bin(number_b)[2:],'= expected:',bin(number_a - number_b)[2:],'got:',bin(samples.data['result'][0])[2:])


# failed
number_a = 1
number_b = 3
samples = non_modular_subtracting(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' - ',bin(number_b)[2:],'= expected:',bin(number_a - number_b)[2:],'got:',bin(samples.data['result'][0])[2:])


# failed
number_a = -3
number_b = 1
samples = non_modular_subtracting(number_a, number_b, False)
print(samples.histogram(key='result'))
print(bin(number_a)[2:],' - ',bin(number_b)[2:],'= expected:',bin(number_a - number_b)[2:],'got:',bin(samples.data['result'][0])[2:])


Counter({1: 1000})
1  -  0 = expected: 1 got: 1
Counter({14: 1000})
1  -  11 = expected: b10 got: 1110
Counter({0: 1000})
b11  -  1 = expected: b100 got: 0


#### possible tests
- come up with similar tests like qFT based addition
    -  try: change: rotation operator
- the problem with negative integers stll is present -> experiment with qFT based two's complement method

## qFT based Multiplication

### qFT based two's complement

In [23]:
class QTwoComplement(cirq.Circuit):
    def __init__(self, register_a, *contents: 'cirq.OP_TREE', debug=False):
        super().__init__(*contents)
        self.register_a = register_a
        self.register_one = [cirq.NamedQubit("q_one")]

    def apply(self):
        self._apply_x_gate(self.register_a)
        self._apply_x_gate(self.register_one)
        self._apply_qmadd()
       # self._apply_measurement()
    
    def _apply_x_gate(self, register):
        for qubit in register:
            self.append(cirq.X(qubit))
    
    def _apply_qmadd(self):
        qmadd = QModularAdder(self.register_a, self.register_one)
        qmadd.apply()
        self.append(qmadd)
    def _apply_measurement(self):
        self.append(cirq.measure(*self.register_a, key='result'))
    

def negation(number_a, print_circuit=False):
    circuit = CircuitInitializer(number_a, 0)

    qnmsub = QTwoComplement(circuit.register_a)
    qnmsub.apply()
    circuit += qnmsub
    
    if print_circuit:
        print(circuit)
    
    samples = simulate(circuit, 1000)
    return samples

In [26]:
# failed
number_a = 1
samples = negation(number_a, False)
print(samples.histogram(key='result'))
#print(bin(number_a)[2:],' expected:',"11",'got:',bin(samples.data['result'][0])[2:])

# failed
number_a = 3
samples = negation(number_a, False)
print(samples.histogram(key='result'))
#print(bin(number_a)[2:],' expected:',"101",'got:',bin(samples.data['result'][0])[2:])


Counter({1: 508, 3: 492})
Counter({7: 451, 1: 401, 3: 77, 5: 71})


Note: implement as operator instead of circuit, since we will need the controlled version of it for the qFT based "abs" operator.


## Modular Adder with cirq ArithmeticOperation class
* https://quantumai.google/cirq/tutorials/shor